In this tutorial, we will show you how to use Haystack to evaluate the performance of a RAG pipeline.

We will use the dataset from the [ARAGOG - Advanced Retrieval Augmented Generation Output Grading (ARAGOG)](https://arxiv.org/pdf/2404.01037) paper.
The dataset is composed of a collection of 13 public AI/LLM-ArXiv research papers and 107 question-answer (QA) pairs. The (QA) pairs generated with the assistance of GPT-4, and then each pair was validated/corrected by humans.

We will use the following Haystack components to evaluate the performance of a RAG pipeline: 

- [ContextRelevance](https://docs.haystack.deepset.ai/docs/contextrelevanceevaluator)
- [Faithfulness](https://docs.haystack.deepset.ai/docs/faithfulnessevaluator)
- [Semantic Answer Similarity](https://docs.haystack.deepset.ai/docs/sasevaluator)


We will build a RAG pipeline and then evaluate it using the ARAGOG dataset by varying three parameters:

- `top_k`: the maximum number of documents returned by the retriever
- `embedding_model`: the model used to encode the documents and the question
- `chunk_size`: the number of tokens in the input text that the model can process at once

More specifically, we will evaluate the pipeline using the following values for each parameter:

```
embedding_models = {
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/msmarco-distilroberta-base-v2",
        "sentence-transformers/all-mpnet-base-v2"
    }

top_k_values = [1, 2, 3]
chunk_sizes = [64, 128, 256]
```

In [14]:
import os
from pathlib import Path

from openai import BadRequestError

from haystack.components.evaluators import ContextRelevanceEvaluator, FaithfulnessEvaluator, SASEvaluator
from haystack.evaluation import EvaluationRunResult


## We need to define a function to load the dataset the questions and answers

In [2]:
import json
from typing import Tuple, List

def read_question_answers() -> Tuple[List[str], List[str]]:
    with open("datasets/ARAGOG/eval_questions.json", "r") as f:
        data = json.load(f)
        questions = data["questions"]
        answers = data["ground_truths"]
    return questions, answers

## We will define an indexing pipeline which depend on two parameters `embedding_model` and the `chunk_size`

In [11]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

def indexing(embedding_model: str, chunk_size: int):
    files_path = "datasets/ARAGOG/papers_for_questions"
    document_store = InMemoryDocumentStore()
    pipeline = Pipeline()
    pipeline.add_component("converter", PyPDFToDocument())
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("splitter", DocumentSplitter(split_length=chunk_size))  # splitting by word
    pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP))
    pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(embedding_model))
    pipeline.connect("converter", "cleaner")
    pipeline.connect("cleaner", "splitter")
    pipeline.connect("splitter", "embedder")
    pipeline.connect("embedder", "writer")
    pdf_files = [files_path+"/"+f_name for f_name in os.listdir(files_path)]
    pipeline.run({"converter": {"sources": pdf_files}})

    return document_store

## We will define a function to run each query over a RAG architecture. This pipeline has the following components:
- ToDo...
- ....
- ...
- 

## Notice that we run the RAG pipeline wrapped within a try/except, to avoid breaking the whole process if at some point there's an error while communicating with OpenAI API.

In [12]:
from architectures.basic_rag import basic_rag

def run_basic_rag(doc_store, sample_questions, embedding_model, top_k):
    """
    A function to run the basic rag model on a set of sample questions and answers
    """

    rag = basic_rag(document_store=doc_store, embedding_model=embedding_model, top_k=top_k)

    predicted_answers = []
    retrieved_contexts = []
    for q in tqdm(sample_questions):
        try:
            response = rag.run(
                data={"query_embedder": {"text": q}, "prompt_builder": {"question": q}, "answer_builder": {"query": q}})
            predicted_answers.append(response["answer_builder"]["answers"][0].data)
            retrieved_contexts.append([d.content for d in response['answer_builder']['answers'][0].documents])
        except BadRequestError as e:
            print(f"Error with question: {q}")
            print(e)
            predicted_answers.append("error")
            retrieved_contexts.append(retrieved_contexts)

    return retrieved_contexts, predicted_answers


## We will define another function to run every predicted answer and ground truth labels through the Evaluators

In [13]:
def run_evaluation(sample_questions, sample_answers, retrieved_contexts, predicted_answers, embedding_model):
    context_relevance = ContextRelevanceEvaluator(raise_on_failure=False)
    faithfulness = FaithfulnessEvaluator(raise_on_failure=False)
    sas = SASEvaluator(model=embedding_model)
    sas.warm_up()

    results = {
        "context_relevance": context_relevance.run(sample_questions, retrieved_contexts),
        "faithfulness": faithfulness.run(sample_questions, retrieved_contexts, predicted_answers),
        "sas": sas.run(predicted_answers, sample_answers),
    }

    inputs = {'questions': sample_questions, "true_answers": sample_answers, "predicted_answers": predicted_answers}

    return results, inputs

## We also need a function to orchestrate everything, indexing, running the dataset over the RAG for each possible parameter combination, and running the evaluation.
## Notice that for each parameter combination, two `.csv` files are generated, one containing the aggregated scores and one the detailed scores.

In [ ]:
def parameter_tuning(questions, answers):
    embedding_models = {
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/msmarco-distilroberta-base-v2",
        "sentence-transformers/all-mpnet-base-v2"
    }
    top_k_values = [1, 2, 3]
    chunk_sizes = [64, 128, 256]

    # create results directory if it does not exist using Pathlib
    out_path = Path("aragog_results")
    out_path.mkdir(exist_ok=True)

    for embedding_model in embedding_models:
        for top_k in top_k_values:
            for chunk_size in chunk_sizes:
                name_params = f"{embedding_model.split('/')[-1]}__top_k:{top_k}__chunk_size:{chunk_size}"
                print(name_params)
                print("Indexing documents")
                doc_store = indexing(embedding_model, chunk_size)
                print("Running RAG pipeline")
                retrieved_contexts, predicted_answers = run_basic_rag(doc_store, questions, embedding_model, top_k)
                print(f"Running evaluation")
                results, inputs = run_evaluation(questions, answers, retrieved_contexts, predicted_answers, embedding_model)
                eval_results = EvaluationRunResult(run_name=name_params, inputs=inputs, results=results)
                eval_results.score_report().to_csv(f"{out_path}/score_report_{name_params}.csv")
                eval_results.to_pandas().to_csv(f"{out_path}/detailed_{name_params}.csv")

# We can then start the whole process

In [ ]:
parameter_tuning(questions, answers)

## After the process is finished we can analyze de results